In [2]:
import numpy as np
from  sklearn.datasets import load_svmlight_file as a
from scipy.sparse import coo_matrix, vstack
from sklearn import svm 
from sklearn import cross_validation 
import matplotlib.pyplot as plt
from sklearn.decomposition import RandomizedPCA
from sklearn import preprocessing
from matplotlib.mlab import PCA as mlabPCA
from pylab import show,title,plot
from sklearn.lda import LDA
%matplotlib inline

In [3]:
#Load all Datasets
Data1=a("C:\Users\Sanju\Desktop\datasets\TV_News_Channel_Commercial_Detection_Dataset\BBC.txt",n_features=4125,dtype=np.float64,multilabel=False,zero_based='auto',query_id=False)
Data2=a("C:\Users\Sanju\Desktop\datasets\TV_News_Channel_Commercial_Detection_Dataset\CNN.txt",n_features=4125,dtype=np.float64,multilabel=False,zero_based='auto',query_id=False)
Data3=a("C:\Users\Sanju\Desktop\datasets\TV_News_Channel_Commercial_Detection_Dataset\CNNIBN.txt",n_features=4125,dtype=np.float64,multilabel=False,zero_based='auto',query_id=False)
Data4=a("C:\Users\Sanju\Desktop\datasets\TV_News_Channel_Commercial_Detection_Dataset\NDTV.txt",n_features=4125,dtype=np.float64,multilabel=False,zero_based='auto',query_id=False)
Data5=a("C:\Users\Sanju\Desktop\datasets\TV_News_Channel_Commercial_Detection_Dataset\TIMESNOW.txt",n_features=4125,dtype=np.float64,multilabel=False,zero_based='auto',query_id=False)

In [4]:
#x is an sparse matrix(n_samples.n_features), (y is a list of n_samples)
data1x,data1y=Data1
data2x,data2y=Data2
data3x,data3y=Data3
data4x,data4y=Data4
data5x,data5y=Data5

x1=coo_matrix(data1x)
x2=coo_matrix(data2x)
x3=coo_matrix(data3x)
x4=coo_matrix(data4x)
x5=coo_matrix(data5x)

data_X=vstack([x1,x2,x3,x4,x5]).toarray()

print data_X.shape

Data_Y=np.concatenate((data1y,data2y,data3y,data4y,data5y), axis=1)
print Data_Y.shape    

(129685L, 4125L)
(129685L,)


In [5]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    data_X, Data_Y, test_size=0.25,random_state=150)

In [6]:
#Normalized Data
from sklearn import preprocessing

#Normalise Each Feature
X_train_1=X_train
X_test_1=X_test

X_train_1=preprocessing.normalize(X_train_1,norm='l2')
X_test_1=preprocessing.normalize(X_test_1,norm='l2')


        
print  X_train_1.shape         
print  X_test_1.shape

(97263L, 4125L)
(32422L, 4125L)


In [12]:
#Lda implementation
lda = LDA(n_components=3)
train_lda=lda.fit_transform(X_train_1[0:75000,], y_train[0:75000])
train_lda.shape

(75000L, 1L)

In [17]:
test_lda=lda.transform(X_test_1[0:25000,])
test_lda.shape

(25000L, 1L)

In [27]:
train_lda_1=train_lda
test_lda_1=test_lda
print train_lda_1.shape
test_lda_1.shape

(75000L, 1L)


(25000L, 1L)

In [18]:
slf=svm.SVC(kernel='linear').fit(train_lda,y_train[0:75000])

In [75]:
slf.score(test_lda,y_test[0:25000])*100

89.459999999999994

In [28]:
ty=slf.support_vectors_
print ty.shape
ay=slf.n_support_
print ay.shape 
ab=slf.coef_
print ab.shape

(20778L, 1L)
(2L,)
(1L, 1L)


In [30]:
from sklearn.ensemble import RandomForestClassifier



In [31]:
clf = RandomForestClassifier(n_estimators=1000,n_jobs=2)
clf.fit(train_lda_1,y_train[0:75000])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [34]:
importances = clf.feature_importances_
importances.shape
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)

indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(train_lda_1.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))


Feature ranking:
1. feature 0 (1.000000)


In [35]:
clf.score(test_lda_1,y_test[0:25000])*100

84.012